<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score


In [ ]:
data = pd.read_csv('/datasets/Churn.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [ ]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [ ]:
data.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

у нас есть категориальные признаки, а есть количественные. и вот тут надо подумать, а как мне вообще модель тренить то. может надо как-то пробежаться только по категориальным признакам сначала, а потом соединить с остальным датасетом? эх

## Исследование задачи

In [ ]:
categories = ['Geography', 'Gender', 'HasCrCard', 'IsActiveMember']

# закодируем категориальные признаки
encoder = OrdinalEncoder()
data[categories] = encoder.fit_transform(data[categories])

<div class="alert alert-success">
<h2> Комментарий ревьюера ✔️ <a class="tocSkip"> </h2>

Здесь круто, идем дальше

In [ ]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,0.0,0.0,42,2.0,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,Hill,608,2.0,0.0,41,1.0,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,Onio,502,0.0,0.0,42,8.0,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,Boni,699,0.0,0.0,39,1.0,0.00,2,0.0,0.0,93826.63,0
4,5,15737888,Mitchell,850,2.0,0.0,43,2.0,125510.82,1,1.0,1.0,79084.10,0


In [ ]:
"""
# стандартизируем количественные признаки 
scaler = StandardScaler()
numeric = ['CreditScore','Age', 
           'Tenure', 'Balance', 'NumOfProducts', 
           'HasCrCard','EstimatedSalary']
scaler.fit(data[numeric])
data[numeric] = scaler.transform(data[numeric]) 
"""

## Борьба с дисбалансом

In [ ]:
data['Exited'].value_counts(normalize=True)

0    0.7963
1    0.2037
Name: Exited, dtype: float64

Видим дисбаланс, будем исправлять.

In [ ]:
target = data['Exited']
features = data.drop(['RowNumber', 'CustomerId', 'Surname','Exited'], axis=1)
features['Tenure'] = features['Tenure'].fillna(0)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [ ]:
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, test_size=0.5,  random_state = 12345)

In [ ]:
# проверим модель без учета дисбаланса
best_model = None
best_depth = 0
best_n_estimators = 0
best_result = 0
for n_est in range(20,80,1):
    for depth in range(1, 18, 1):
        model = RandomForestClassifier(n_estimators=n_est, max_depth=depth, random_state=12345)
        model.fit(features_train, target_train)
        prediction = f1_score(target_valid, model.predict(features_valid))
        if prediction>best_result:
            best_model = model
            best_depth = depth
            best_n_estimators = n_est
            best_result = prediction
                
print(best_n_estimators, best_depth)
print("F1:", best_result)
auc_roc = roc_auc_score(target_valid,best_model.predict(features_valid))
print("AUC-ROC", auc_roc)

35 17
F1: 0.6023529411764705
AUC-ROC 0.7261220967636306


Обучим модель и сравним лучшие параметры и результат.

In [ ]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    
    target_zeros = target[target == 0]
    target_ones = target[target ==1]
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled = shuffle(features_upsampled,random_state=12345)
    target_upsampled = shuffle(target_upsampled,random_state=12345)

    return features_upsampled, target_upsampled

In [ ]:
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)


In [ ]:
target_upsampled.value_counts(normalize=True)

1    0.500416
0    0.499584
Name: Exited, dtype: float64

In [ ]:
best_model = None
best_depth = 0
best_n_estimators = 0
best_result = 0
for n_est in range(20,80,1):
    for depth in range(1, 18, 1):
        model = RandomForestClassifier(n_estimators=n_est, max_depth=depth, random_state=12345)
        model.fit(features_upsampled, target_upsampled)
        prediction = f1_score(target_valid, model.predict(features_valid))
        if prediction>best_result:
            best_model = model
            best_depth = depth
            best_n_estimators = n_est
            best_result = prediction
                
print(best_n_estimators, best_depth)
print("F1:", best_result)
auc_roc = roc_auc_score(target_valid,best_model.predict(features_valid))
print("AUC-ROC", auc_roc)

78 11
F1: 0.6499102333931777
AUC-ROC 0.7868622896128761


In [ ]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction,random_state=12345)] + [target_ones])
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = shuffle(target_downsampled, random_state=12345)
    features_downsampled = shuffle(features_downsampled, random_state=12345)
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

In [ ]:
target_downsampled.value_counts(normalize=True)

1    0.500333
0    0.499667
Name: Exited, dtype: float64

In [ ]:
dbest_model = None
best_depth = 0
best_n_estimators = 0
best_result = 0
for n_est in range(30,80,1):
    for depth in range(1, 18, 1):
        model = RandomForestClassifier(n_estimators=n_est, max_depth=depth, random_state=12345)
        model.fit(features_downsampled, target_downsampled)
        prediction = f1_score(target_valid, model.predict(features_valid))
        if prediction>best_result:
            dbest_model = model
            best_depth = depth
            best_n_estimators = n_est
            best_result = prediction

                
print(best_n_estimators, best_depth)
print("F1:", best_result)
auc_roc = roc_auc_score(target_valid,dbest_model.predict(features_valid))
print("AUC-ROC:", auc_roc)

32 6
F1: 0.6222222222222222
AUC-ROC: 0.7969092499065802


<span style="color:blue"> Результаты после downsampling хуже. Возьмем модель после upsampling и протестируем.


## Тестирование модели

In [ ]:
prediction = best_model.predict(features_test)
print("F1:", f1_score(target_test, prediction))
print("AUC-ROC:", roc_auc_score(target_test, prediction))

F1: 0.6114864864864865
AUC-ROC: 0.7616571935522676


Протестировали модель без учета дисбалланса и получили следующие результаты метрик:
* F1: 0.6076555023923444
* AUC-ROC 0.7272604697570315

После балансировки классов upsampling метрики выросли:
* F1: 0.6495049504950495
* AUC-ROC: 0.7722733944317959

После балансировки классов методом downsampling метрики упали
* F1: 0.6222222222222222
* AUC-ROC: 0.7969092499065802

При тестировании лучшей модели получили следующие результаты:
* F1: 0.5931558935361216
* AUC-ROC: 0.7366624563936004

<span style="color:blue"> Без стандартизации количественных признаков довели F1 до 0.61.